In [79]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password1"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.fetch_all({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))


# Rescue type queries
rescue_options = ['All', 'Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking']
water_rescue_query = {"breed":{"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},"sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
mountain_wilderness_query = {"breed":{"$in": ["German Shephard","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
disaster_rescue_query = {"breed":{"$in": ["Doberman Pinscher","German Shephard","Golden Retriever","Bloodhound","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}


keys = ['All'] + db.collection.distinct("animal_type")
breed_options = {}
type_options = {}
for k in keys:
    breed_options.update({k : ['All'] + db.collection.distinct("breed",{"animal_type":k})})
    type_options.update({k : ['All'] + db.collection.distinct("outcome_type",{"animal_type":k})})
    

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),    
    html.Center([html.B(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'maxWidth': '20%', 'maxHeight': '20%', 'objectFit': 'contain'}),
                       ),html.H3('CS-340 Dashboard - Travis Williamson', style={"whiteSpace":"nowrap","width":"auto"})]),    
    html.Hr(),
    html.Div(className="filterRow",
             style={'display': 'flex',
                    'justifyContent': 'left'
                   },
             children=[
                 
                html.P("Rescue Type: ",style={
                                'display': 'inline',
                                "whiteSpace": "nowrap"
                              }
                      ),
                 dcc.Dropdown(id='rescuetype',
                              options=rescue_options,
                              value = 'All',
                              style={
                                'display': 'inline-block',
                                "width": "100%",
                                "margin-left": "0.5%"
                              },
                              clearable = False
                             ),
                html.P(" OR ",style={
                                'display': 'inline-block',                                  
                                "margin-left":  "3%",
                                "whiteSpace": "nowrap"
                              }),                 
                 html.P("Custom Filter:",style={
                                'display': 'inline-block',                                  
                                "margin-left":  "3%",
                                "whiteSpace": "nowrap"
                              }),
                 dcc.Dropdown(id='dropdownfilter', 
                              options=[{'label':k,'value':k} for k in breed_options.keys()],
                              value='All',
                              style={
                                'display': 'inline-block',
                                'width': '100%',
                                "margin-left":  "0.5%"                            
                              },
                              clearable = False,
                             ),
                 dcc.Dropdown(id='breeddropdownfilter',
                              value = 'All',
                              style={
                                'display': 'inline-block',
                                'width': '100%',                                  
                                "margin-left":  "3%"                              
                              },
                              clearable = False,
                             ),
                 dcc.Dropdown(id='outcomedropdownfilter',
                              value = 'All',
                              style={
                                'display': 'inline-block',
                                'width': '95%',                                  
                                "margin-left":  "3%"                                  
                              },
                              clearable = False,)
            ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                        editable=False,
                        filter_action="native",
                        sort_action="native",
                        sort_mode="multi",
                        column_selectable=False,
                        row_selectable="single",
                        row_deletable=False,
                        selected_columns=[],
                        selected_rows=[0],
                        page_action="native",
                        page_current=0,
                        page_size=10

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

#Add options to breed drop down box
@app.callback(
    Output('breeddropdownfilter','options'),
    [Input('dropdownfilter', 'value')])
def set_breed_options(selected_animal):
    return [{'label': i, 'value': i} for i in breed_options[selected_animal]]

#Add options to outcome type drop down box
@app.callback(
    Output('outcomedropdownfilter','options'),
    [Input('dropdownfilter', 'value')])
def set_outcome_options(selected_animal):
    return [{'label': i, 'value': i} for i in type_options[selected_animal]]

@app.callback(
    Output('datatable-id','data'),
    [Input('breeddropdownfilter','value')],
    [Input('dropdownfilter', 'value')],
    [Input('outcomedropdownfilter', 'value')],
    [Input('rescuetype', 'value')])
def set_filter(breed,animal_type,outcome_type,rescuefilter):
    # grab all records
    df = pd.DataFrame.from_records(db.fetch_all({}))
    
    if rescuefilter == 'All':
        if breed != 'All':
            df = df[df['breed'] == breed]
        
        if animal_type != 'All':
            df = df[df['animal_type'] == animal_type]
        
        if outcome_type != 'All':
            df = df[df['outcome_type'] == outcome_type]
    else:
        if rescuefilter == "Water Rescue":
            df = pd.DataFrame.from_records(db.fetch_all(water_rescue_query))
        elif rescuefilter == "Mountain or Wilderness Rescue":
            df = pd.DataFrame.from_records(db.fetch_all(mountain_wilderness_query))
        elif rescuefilter == "Disaster or Individual Tracking":
            df = pd.DataFrame.from_records(db.fetch_all(disaster_rescue_query))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    df.drop(columns=['_id'],inplace=True)
    data=df.to_dict('records')
    return data                                     

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(Output('graph-id', "children"),
             [Input('datatable-id','derived_virtual_data')])
def update_graphs(viewData):
    if viewData is None:
        return    
        
    derived_df = pd.DataFrame.from_records(viewData)
    
    derived_df['Percentage'] = 1.0 * derived_df.groupby('breed')['breed'].transform('count') / len(derived_df)
    
    for row in derived_df.itertuples():
        if derived_df.at[row.Index, 'Percentage'] < 0.015:
            derived_df.at[row.Index, 'breed'] = "Other"
        if derived_df.at[row.Index, 'breed'] is None:
            derived_df.at[row.Index, 'breed'] = "Other"
    
    return [
        dcc.Graph(
            figure = px.pie(derived_df,names='breed', title='Preferred Animals').update_traces(textposition='inside', textinfo='percent+label')            
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://julietmuseum-pearlsardine-3000.codio.io/proxy/8050/
